In [13]:
%config InteractiveShell.ast_node_interactivity = 'all' # 'all', 'last', 'last_expr' or 'none', 'last_expr_or_assign'
# %config InlineBackend.figure_format = 'svg'


In [2]:
import numpy as np
import gym
from envs.cleanup import CleanupEnv
from envs.cleanup import CleanupAgent

# self, agent_id, start_pos, start_orientation, grid, row_size, col_size

env = 'cleanup-v0'
try:
    gym.register(env, entry_point=CleanupEnv)
except:
    raise Exception('environment registered')
env = gym.make(env)



In [3]:
a = env.action_space.sample()
a

0

In [6]:
s = env.reset()
s['agent-0'].shape

(15, 15, 3)

In [19]:
env.action_space
env.observation_space

Discrete(9)

Box(15, 15, 3)

In [24]:
from envs.harvest import HarvestEnv
from envs.cleanup import CleanupEnv
from algo.ppo import PPO
from algo.model import Policy
from algo.storage import RolloutStorage
# from config import get_config
from envs.vec_env import SubprocVecEnv, DummyVecEnv
from algo.algo_utils import update_linear_schedule

In [25]:
def make_parallel_env(args):
    def get_env_fn(rank):
        def init_env():
            if args['env_name'] == "cleanup":
                env = CleanupEnv(num_agents=args.num_agents)
            elif args['env_name'] == "harvest":
                env = HarvestEnv(num_agents=args.num_agents)
            else:
                print("Can not support the " + args['env_name'] + "environment." )
                raise NotImplementedError
            env.seed(args['seed'] + rank * 1000)
            return env
        return init_env
    if args['n_rollout_threads'] == 1:
        return DummyVecEnv([get_env_fn(0)])
    else:
        return SubprocVecEnv([get_env_fn(i) for i in range(args['n_rollout_threads'])])


In [28]:
args = {'env_name':'harvest', 'seed':2, 'n_rollout_threads':2}
args.n_rollout_threads
env = make_parallel_env(args)

AttributeError: 'dict' object has no attribute 'n_rollout_threads'